# A Simple Backtesting Tool

## Housekeeping

In [1]:
from simfin.names import *
import pandas as pd
import numpy as np
import seaborn as sns
import time

# Import the main functionality from the SimFin Python API.
import simfin as sf
import pandas_datareader.data as web

import warnings
warnings.filterwarnings('ignore')

api_key = 'free'
sf.set_api_key(api_key)

root = '/Users/Wanderer/Desktop/hu/Report/' # Change this when running locally
sf.set_data_dir(root + 'simfin_data/')

bt_log = pd.DataFrame()
        
pd.set_option('display.max_columns', None)

## Load Data

In [48]:
# Set testing and training endpoints:
pre = '2015-01-01'
mid = '2019-01-01'
post = '2019-12-31'
bt_bracket = 20

def timecheck(price):
    '''
    Input: (DataFrame) data of one ticker only.
    Output: (Boolean) True only if data begin earlier than 2015-01-01 and end later than 2019-12-31.
    '''
    precheck = (price.index[0] <= pd.to_datetime(pre , format = '%Y-%m-%d'))
    postcheck = (price.index[-1] >= pd.to_datetime(post , format = '%Y-%m-%d'))
    check = precheck & postcheck
    return check

def extracttrain(df):
    out = df[df.index >= pd.to_datetime(pre , format = '%Y-%m-%d')]
    out = out[out.index < pd.to_datetime(mid , format = '%Y-%m-%d')]
    return out

def extracttest(df):
    out = df[df.index >= pd.to_datetime(mid , format = '%Y-%m-%d')]
    out = out[out.index <= pd.to_datetime(post , format = '%Y-%m-%d')]
    return out

def tickerfilter(prices, name):
    '''
    Input1: (DataFrame) Data of multple tickers.
    Input2: (str) name of the indicator.
    Output: Writes pickle: filtered Input1
    
        
    pre to mid: training
    mid to post: testing
    '''
    good = sf.apply(df = prices, func = timecheck)
    good = good[good]
    good = list(good.index)
    df = prices.loc[good]
    df.to_pickle(root + 'good' + str(name) + '.pkl')
    
    train = sf.apply(df = df, func = extracttrain)
    test = sf.apply(df = df, func = extracttest)
    
    train.to_pickle(root + 'train' + str(name) + '.pkl')
    test.to_pickle(root + 'test' + str(name) + '.pkl')
    print(name + ' saved!')
    return 0

def reloaddata():
    '''
    Loads data from SimFin.com or from local disk.
    Writes pickle
    '''
    hub = sf.StockHub(market='us',
                  refresh_days=60,
                  refresh_days_shareprices=30)
    
    prices = hub.load_shareprices(variant = 'daily')
    incomes = hub.load_income(variant = 'ttm')
    balances = hub.load_balance(variant = 'ttm')
    cashflows = hub.load_cashflow(variant = 'ttm')
    
    fin_signals = hub.fin_signals(variant='daily')
    grow_signals = hub.growth_signals(variant='daily')
    
    
    
    tickerfilter(prices, 'prices')
    tickerfilter(incomes, 'incomes')
    tickerfilter(balances, 'balances')
    tickerfilter(cashflows, 'cashflows')
    tickerfilter(fin_signals, 'finsig')
    tickerfilter(grow_signals, 'growsig')
    
##================================##
# Last refresh on **20210614**
# Set to True on a new device / when data is too old.
if True:
    reloaddata()
##================================##

Dataset "us-shareprices-daily" on disk (0 days old).
- Loading from disk ... Done!
Dataset "us-income-ttm" on disk (2 days old).
- Loading from disk ... Done!
Dataset "us-balance-ttm" on disk (2 days old).
- Loading from disk ... Done!
Dataset "us-cashflow-ttm" on disk (2 days old).
- Loading from disk ... Done!
Cache-file 'fin_signals-7dc370f1.pickle' on disk (0 days old).
- Loading from disk ... Done!
Dataset "us-income-quarterly" on disk (0 days old).
- Loading from disk ... Done!
Dataset "us-balance-quarterly" on disk (0 days old).
- Loading from disk ... Done!
Dataset "us-cashflow-quarterly" on disk (0 days old).
- Loading from disk ... Done!
Cache-file 'growth_signals-7dc370f1.pickle' on disk (0 days old).
- Loading from disk ... Done!
prices saved!
incomes saved!
balances saved!
cashflows saved!
finsig saved!
growsig saved!


## Functions

### Backtesting

In [49]:
# A helper function that returns the list of tickers whose average adj. close prices are at least $5.
## Used in signal calculating.
def overfive():
    df = pd.read_pickle(root + 'trainprices.pkl')
    include = df.groupby(['Ticker'])['Adj. Close'].mean()
    include = include[include >= 5]
    include = list(include.index)
    return include

# Backtesting:
def topandbottom(df, text):
    bracket = bt_bracket
    winner = df[-bracket:]
    loser = df[:bracket]
    test = pd.read_pickle(root + 'testprices.pkl')
    badstock = []
    for stock in winner.index:
        try:
            winner.loc[stock,'annualret'] = test.loc[stock].iloc[-1]['Adj. Close'] / test.loc[stock].iloc[0]['Adj. Close'] - 1
        except:
            badstock.append(stock)
    for stock in loser.index:
        try:
            loser.loc[stock,'annualret'] = test.loc[stock].iloc[-1]['Adj. Close'] / test.loc[stock].iloc[0]['Adj. Close'] - 1
        except:
            badstock.append(stock)
    winner_yield = winner['annualret'].mean()
    loser_yield = loser['annualret'].mean()
    diff_yield = winner_yield - loser_yield
    data = [[text, winner_yield, loser_yield, diff_yield, pre, mid, post, bracket]]
    add = pd.DataFrame(data, columns = ['Description','Top yield','Bottom yield','Yield difference',
                                       'Training start','Training end','Test end','Bracket'])
    
    global bt_log
    bt_log = pd.concat([bt_log, add])
    print(badstock)
    print('According to {}, \nthe top 100 stocks yield annually {:.4f};\nthe bottom 100 stocks yield annually {:.4f}'.format(text,winner_yield,loser_yield))
    

def bt_prices(func, text):
    '''
    func: (Function) A function whose input is the price data for one ticker only.
    name: (str) Name of the factor 
    '''
    prices = pd.read_pickle('trainprices.pkl')
    factors = sf.apply(df = prices, func = func)
    factor = factors.groupby(['Ticker'])[factors.columns[-1]].mean().sort_values().dropna()
    factor = pd.DataFrame(factor)
    
    topandbottom(factor, text)


def bt_prices_grp(func, text):
    '''
    func: (Function) A function whose input is the price data for all tickers (GRouPed, that is).
    name: (str) Name of the factor 
    '''
    prices = pd.read_pickle('trainprices.pkl')
    factors = func(prices)
    factor = factors.groupby(['Ticker'])[factors.columns[-1]].mean().sort_values().dropna()
    factor = pd.DataFrame(factor)
    
    topandbottom(factor, text)
    
def bt_incomes(func, text):
    '''
    func: (Function) A function whose input is the price data for one ticker only.
    name: (str) Name of the factor 
    '''
    incomes = pd.read_pickle('trainincomes.pkl')
    factors = func(incomes)
    factor = factors.groupby(['Ticker'])[factors.columns[-1]].mean().sort_values().dropna()
    factor = pd.DataFrame(factor)
    
    topandbottom(factor, text)    

def bt_signals(func, text):
    '''
    func: (Function) A function whose input is a financial/growth signal, for all tickers.
    name: (str) Name of the factor 
    '''
    factors = func()
    factor = factors.groupby(['Ticker'])[factors.columns[-1]].mean().sort_values().dropna()
    factor = pd.DataFrame(factor)
    
    topandbottom(factor, text)
    
def bt_prices_incomes(func, text):
    '''
    func: (Function) A function whose input is prices and incomes, for all tickers.
    name: (str) Name of the factor 
    '''
    prices = pd.read_pickle(root + 'trainprices.pkl')
    incomes = pd.read_pickle(root + 'trainincomes.pkl')
    factors = func(prices, incomes)
    factor = factors.groupby(['Ticker'])[factors.columns[-1]].mean().sort_values().dropna()
    factor = pd.DataFrame(factor)
    
    topandbottom(factor, text)
    
def bt_cashflows_incomes(func, text):
    '''
    func: (Function) A function whose input is cashflows and incomes, for all tickers.
    name: (str) Name of the factor 
    '''
    cashflows = pd.read_pickle(root + 'traincashflows.pkl')
    incomes = pd.read_pickle(root + 'trainincomes.pkl')
    factors = func(cashflows, incomes)
    factor = factors.groupby(['Ticker'])[factors.columns[-1]].mean().sort_values().dropna()
    factor = pd.DataFrame(factor)
    
    topandbottom(factor, text)

def bt_prices_balances(func, text):
    '''
    func: (Function) A function whose input is prices and balances, for all tickers.
    name: (str) Name of the factor 
    '''
    prices = pd.read_pickle(root + 'trainprices.pkl')
    balances = pd.read_pickle(root + 'trainbalances.pkl')
    factors = func(prices, balances)
    factor = factors.groupby(['Ticker'])[factors.columns[-1]].mean().sort_values().dropna()
    factor = pd.DataFrame(factor)
    
    topandbottom(factor, text)

### Logging

In [50]:
def bt_reset():
    bt_log = pd.DataFrame()
def bt_print():
    bt_log.sort_values(by = ['Description','Bracket','Training start','Training end','Test end'],
                       axis = 0, inplace = True)
    bt_log.to_csv(root + 'backtesting_results.csv', index = False)  

## Factors

### Prices
For single-ticker functions, test with br_prices( );  
br_prices_grp( ) for all-tickers operations.

In [51]:
def logprice(prices):
    '''
    Price
    Blume & Husic (JF 1972)
    '''
    logprice = pd.DataFrame()
    logprice['logprice'] = prices['Adj. Close'].apply(lambda x: np.log(x))
    prices = pd.concat([prices, logprice], axis = 1, join = 'inner')
    return prices

bt_prices(logprice, 'Price: Blume & Husic (JF 1972)')

# Long-term reversal is unavailable for any two-year training dataset 
## for the obvious reason that two years is not really "long term".
'''
def longreversal(prices):
    ''''''
    Long-term Reversal
    Debondt and Thaler (JF 1985)
    ''''''
    longrev = sf.asfreq(df = prices, freq = 'D', method = 'ffill')
    longrev['longreversal'] = longrev['Adj. Close'].pct_change(1410).shift(390)
    longrev = longrev['longreversal']
    prices = pd.concat([prices, longrev], axis = 1, join = 'inner')
    return prices

bt_prices(longreversal, 'Long-term Reversal: Debondt and Thaler (JF 1985)')
'''

def shortreversal(prices):
    '''
    Short-term Reversal
    Jegadeesh (1989)
    '''
    shortrev = sf.asfreq(df = prices, freq = 'D', method = 'ffill')
    shortrev['shortreversal'] = prices['Adj. Close'].pct_change(30)
    shortrev['shortreversal_ex'] = (prices['Adj. Close'].mean() < 5)
    shortrev = shortrev[['shortreversal','shortreversal_ex']]
    shortrev = shortrev[shortrev['shortreversal_ex'] == False]
    prices = pd.concat([prices, shortrev['shortreversal']], axis = 1, join = 'inner')
    return prices

bt_prices(shortreversal, 'Short-term Reversal: Jegadeesh (1989)')

def yearhigh(prices):
    '''
    52-Week High
    George & Hwang (JF 2004)
    '''
    pricefilled = sf.asfreq(df = prices, freq='D', method='ffill')
    pricefilled.rename(columns = {'Adj. Close':'yearhigh'}, inplace = True)
    yearhigh = pricefilled['yearhigh'].rolling(window = '364D').max()
    prices = pd.concat([prices, yearhigh], axis = 1, join = 'inner')
    return prices

bt_prices(yearhigh, '52-Week High: George & Hwang (JF 2004)')

def monthmax(prices):
    '''
    Max
    Bali et al. (JF 2010)
    '''
    pricefilled = sf.asfreq(df = prices, freq = 'D', method = 'ffill')
    pricefilled.rename(columns = {'Adj. Close':'max'}, inplace = True)
    returns = pricefilled['max'].pct_change()
    maxreturns = returns.rolling(window = '30D').max()
    prices = pd.concat([prices, maxreturns], axis = 1, join = 'inner')
    return prices


bt_prices(monthmax, 'Max: Bali et al. (JF 2010)')

def amihud(prices):
    '''
    Amihud's Measure
    Amihud (JFM 2002)
    '''
    amihud = pd.DataFrame()
    amihud['raw'] = (prices['Adj. Close'].diff()) * prices['Volume'] * prices['Adj. Close']
    amihud['Amihud'] = amihud['raw'].rolling(window = "365D").mean()
    amihud['Amihud_ex'] = (prices['Adj. Close'].mean() < 5)
    amihud = amihud[['Amihud','Amihud_ex']]
    amihud = amihud[amihud['Amihud_ex'] == False]
    prices = pd.concat([prices, amihud['Amihud']], axis = 1, join = 'inner')
    return prices

bt_prices(amihud, 'Amihud\'s Measure: Amihud (JFM 2002)')

def equitysize(prices):
    '''
    Size
    Banz (JFE 1981)
    (Or market capitalization)
    '''
    eqsize = pd.DataFrame()
    eqsize['size'] = prices['Adj. Close'] * prices['Shares Outstanding']
    eqsize['size'] = eqsize['size'].apply(lambda x: np.log(x))
    prices = pd.concat([prices, eqsize], axis = 1, join = 'inner')
    return prices

bt_prices(equitysize, 'Size: Banz (JFE 1981)')



[]
According to Price: Blume & Husic (JF 1972), 
the top 100 stocks yield annually -0.0113;
the bottom 100 stocks yield annually 8.9978
[]
According to Short-term Reversal: Jegadeesh (1989), 
the top 100 stocks yield annually 0.3633;
the bottom 100 stocks yield annually -0.3060
[]
According to 52-Week High: George & Hwang (JF 2004), 
the top 100 stocks yield annually -0.0622;
the bottom 100 stocks yield annually 7.9244
[]
According to Max: Bali et al. (JF 2010), 
the top 100 stocks yield annually 0.0976;
the bottom 100 stocks yield annually 5.5009
[]
According to Amihud's Measure: Amihud (JFM 2002), 
the top 100 stocks yield annually -0.1688;
the bottom 100 stocks yield annually 0.0080
[]
According to Size: Banz (JFE 1981), 
the top 100 stocks yield annually 0.3136;
the bottom 100 stocks yield annually 6.8649


In [52]:
# An exception, in that it does not actually use the training data, 
## but records the "earliest observation" in the whole dataset.
## The input, therefore, is redundant in itself, but necessary for fitting into the backtesting function.
def age(prices):
    '''
    Firm Age
    Barry and Brown (JFE 1984)
    '''
    df = pd.read_pickle(root + 'goodprices.pkl')
    def getage(price):
        birthday = price.index[0]
        output = -12 * (birthday.year - 2015) - (birthday.month)
        if price['Adj. Close'].mean() < 5:
            output = np.NaN
        output = pd.DataFrame([[output]])
        return output
    out = sf.apply(df = df, func = getage)
    out = out.reset_index(level = 1)
    return out

bt_prices_grp(age, 'Firm Age: Barry and Brown (JFE 1984)')

[]
According to Firm Age: Barry and Brown (JFE 1984), 
the top 100 stocks yield annually 0.1222;
the bottom 100 stocks yield annually 0.2477


### Signals
Test with bt_signals( )

In [53]:
def assetturnover():
    '''
    Asset Turnover
    Soliman (AR 2008)
    '''
    include = overfive()
    finsig = pd.read_pickle(root + 'trainfinsig.pkl')
    out = finsig.loc[include]
    out = out[['Asset Turnover']]
    return out

bt_signals(assetturnover, 'Asset Turnover: Soliman (AR 2008)')

def assetgrowth():
    '''
    Asset Growth
    Cooper et al. (JF 2008)
    '''
    growsig = pd.read_pickle(root + 'traingrowsig.pkl')
    out = growsig[['Assets Growth']]
    return out

bt_signals(assetgrowth, 'Asset Growth: Cooper et al. (JF 2008)')

def salesgrowth():
    '''
    Sales Growth
    LSV (JF 1994)
    '''
    growsig = pd.read_pickle(root + 'traingrowsig.pkl')
    out = growsig[['Sales Growth']]
    return out

bt_signals(salesgrowth, 'Sales Growth: LSV (JF 1994)')

def profitmargin():
    '''
    Profit Margin
    Soliman (AR 2008)
    '''
    include = overfive()
    finsig = pd.read_pickle(root + 'trainfinsig.pkl')
    out = finsig.loc[include]
    out = out[['Net Profit Margin']]
    return out

bt_signals(profitmargin, 'Profit Margin: Soliman (AR 2008)')

def roa():
    '''
    Return on Assets
    DuPont
    '''
    include = overfive()
    finsig = pd.read_pickle(root + 'trainfinsig.pkl')
    out = finsig.loc[include]
    out = out[['Return on Assets']]
    return out

bt_signals(roa, 'Return on Assets: DuPont')

def roe():
    '''
    Return on Equity
    Haugen and Baker (JFE 1996)
    '''
    include = overfive()
    finsig = pd.read_pickle(root + 'trainfinsig.pkl')
    out = finsig.loc[include]
    out = out[['Return on Equity']]
    return out

bt_signals(roe, 'Return on Equity: Haugen and Baker (JFE 1996)')



[]
According to Asset Turnover: Soliman (AR 2008), 
the top 100 stocks yield annually 0.0821;
the bottom 100 stocks yield annually 0.1573
[]
According to Asset Growth: Cooper et al. (JF 2008), 
the top 100 stocks yield annually 0.2780;
the bottom 100 stocks yield annually 0.2578
[]
According to Sales Growth: LSV (JF 1994), 
the top 100 stocks yield annually 0.5321;
the bottom 100 stocks yield annually 0.1388
[]
According to Profit Margin: Soliman (AR 2008), 
the top 100 stocks yield annually 0.4403;
the bottom 100 stocks yield annually 0.2057
[]
According to Return on Assets: DuPont, 
the top 100 stocks yield annually 0.3445;
the bottom 100 stocks yield annually 0.0431
[]
According to Return on Equity: Haugen and Baker (JFE 1996), 
the top 100 stocks yield annually 0.0509;
the bottom 100 stocks yield annually 0.2591


### Incomes
Test with bt_incomes( )

In [54]:
def earnings_consistency(incomes):
    '''
    Earnings Consistency
    Alwathainani (BAR 2009)
    '''
    include = overfive()
    income = incomes[incomes.index.get_level_values('Ticker').isin(include)]
    income.loc[:,'Earnings per Share'] = income['Net Income (Common)'] / income['Shares (Diluted)']
    income.loc[:,'Earnings Consistency'] = income['Earnings per Share'].diff() / (abs(income['Earnings per Share'].shift(1)) + abs(income['Earnings per Share'].shift(2)) / 2)
    output = income[['Earnings Consistency']]
    return output

bt_incomes(earnings_consistency, 'Earnings Consistency, Alwathainani (BAR 2009)')

[]
According to Earnings Consistency, Alwathainani (BAR 2009), 
the top 100 stocks yield annually 0.2333;
the bottom 100 stocks yield annually -0.2174


### Cashflows and Incomes
Test with bt_cashflows_incomes( )

In [55]:
def investment(cashflows, incomes):
    '''
    Investment
    Titman, Wei, and Xie (JFQA 2004)
    '''
    df = pd.merge(cashflows[CAPEX],
                  incomes[REVENUE],
                  on=['Ticker','Report Date'])
    df['investment'] = df[CAPEX]/df[REVENUE]
    return df

bt_cashflows_incomes(investment, 'Investment: Titman, Wei, and Xie (JFQA 2004)')

['UPL', 'CRR', 'CHK', 'RHE', 'CMBG']
According to Investment: Titman, Wei, and Xie (JFQA 2004), 
the top 100 stocks yield annually 0.0491;
the bottom 100 stocks yield annually 0.2067


### Prices and Balances
Test with bt_prices_balances( )

In [56]:
def leverage(prices, balances):
    '''
    Leverage
    Bhandari (JFE 1988)
    '''
    # Log of Long Term Debt
    balance=balances.fillna(1)
    balance = balances.fillna(1)
    balance['Log of Long Term Debt'] = np.log(balance['Long Term Debt'])

    balance = balance.reset_index(level=1)
    balance = balance.rename(columns = {'Report Date':'Date'})

    # Market Value of Equity
    price = prices.reset_index(level = 1)
    price['Market Value of Equity'] = price['Adj. Close'] * price['Shares Outstanding']

    df = pd.merge(balance[['Date','Log of Long Term Debt']],
                  price[['Date','Market Value of Equity']],
                  on=['Ticker','Date'])

    df['Leverage'] = df['Log of Long Term Debt'] / df['Market Value of Equity']
    
    return df

bt_prices_balances(leverage, 'Leverage: Bhandari (JFE 1988)')

[]
According to Leverage: Bhandari (JFE 1988), 
the top 100 stocks yield annually 0.1593;
the bottom 100 stocks yield annually 0.0894


### Prices and Incomes
Test with bt_prices_incomes( )

In [57]:
def cashflowvar(prices, incomes):
    '''
    Cash Flow Variance
    Haugen and Baker (JFE 1996)
    '''
    
    # Market Value of Equity
    include = overfive()
    price = prices.loc[include]
    price = price.reset_index(level=0)
    price = price.groupby(['Ticker']).resample('Q').mean()
    price['Market Value of Equity'] = price['Adj. Close'] * price['Shares Outstanding']
    price = price.reset_index(level=1)
    
    # Cash Flow Variance
    income = incomes.reset_index(level=1)
    income = income.fillna(0)
    income = income.rename(columns = {'Report Date':'Date'})

    cashflow = pd.merge(price[['Date','Market Value of Equity']],
                         income[['Date','Depreciation & Amortization','Net Income']],
                         on=['Ticker','Date'])
    cashflow['Cash Flow Variance'] = (cashflow['Net Income'] + cashflow['Depreciation & Amortization']) / cashflow['Market Value of Equity']
    cashflow['Ticker'] = cashflow.index
    cashflow.set_index('Date', drop=True, append=False, inplace=True)
    cashflow = cashflow.groupby('Ticker').resample('Y').var()
    
    return cashflow[['Cash Flow Variance']]

bt_prices_incomes(cashflowvar, 'Cash Flow Variance: Haugen and Baker (JFE 1996)')




[]
According to Cash Flow Variance: Haugen and Baker (JFE 1996), 
the top 100 stocks yield annually 0.2513;
the bottom 100 stocks yield annually 0.3213


In [58]:
def Share_Volume(volume):
    volume = volume.resample('M').mean()
    volume['Share_Volume'] = volume['Volume'].rolling(window=3).mean()
    return volume

bt_prices(Share_Volume, 'Share volume')

[]
According to Share volume, 
the top 100 stocks yield annually 0.4080;
the bottom 100 stocks yield annually 5.6639


In [59]:
bt_print()